In [1]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install wget

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Download the library from the following link:
# https://artint.info/AIPython/

from os.path import exists
from pathlib import Path
from typing import Final
from zipfile import ZipFile

from wget import download

URL: Final[str] = 'https://artint.info/AIPython/aipython.zip'
DESTINATION: Final[Path] = Path('aipython.zip')

if not exists(DESTINATION):
    download(url=URL, out=DESTINATION.absolute().as_posix())

if not exists(DESTINATION.stem):
    with ZipFile(DESTINATION, 'r') as zip_ref:
        zip_ref.extractall()

In [3]:
import sys

# Cheat so I don't have to make `aipython` a package with setuptools ....
sys.path.append(DESTINATION.stem)

In [4]:
from random import seed

seed(42)

In [5]:
import pandas as pd

data: pd.DataFrame = pd.DataFrame(
    columns=['problem', 'used_heuristic', 'elapsed_time']
)

In [6]:
#;;
#;; Air cargo transport problem.
#;; http://www.inf.unibz.it/~tessaris/teaching/AI_06-07/labs/2007-01-10/planning.html
#;;

#(define (domain air-cargo)
#  (:requirements :strips)
#  (:predicates (In ?obj ?place)
#           (At ?obj ?place)
#           (Cargo ?obj)
#           (Plane ?obj)
#           (Airport ?obj))

#  (:action LOAD
#     :parameters (?c ?p ?a)
#     :precondition (and (At ?c ?a) (At ?p ?a)
#             (Cargo ?c) (Plane ?p) (Airport ?a))
#     :effect (and (In ?c ?p) (not (At ?c ?a))))
#  
#  (:action UNLOAD
#     :parameters (?c ?p ?a)
#     :precondition (and (In ?c ?p) (At ?p ?a)
#             (Cargo ?c) (Plane ?p) (Airport ?a))
#     :effect (and (At ?c ?a) (not (In ?c ?p))))
#  
#  (:action FLY
#     :parameters (?p ?from ?to)
#     :precondition (and (At ?p ?from)
#             (Plane ?p) (Airport ?from) (Airport ?to))
#     :effect (and (At ?p ?to) (not (At ?p ?from))))
#  )

In [7]:
# STRIPS domain - Air Cargo Transportation
from stripsProblem import Strips, STRIPS_domain

def In(obj: str) -> str:
    return obj + '_is_in'

def At(obj: str) -> str:
    return obj + "_is_at"

def Cargo(obj: str) -> str:
    return obj + "_is_cargo"

def Plane(obj: str) -> str:
    return obj + "_is_plane"

def Airport(obj: str) -> str:
    return obj + "_is_airport"

def Load(cargo: str, plane: str, airport: str) -> str:
    return "load_" + cargo + "_from_" + airport + "_to_" + plane

def Unload(cargo: str, plane: str, airport: str) -> str:
    return "unload_" + cargo + "_from_" + plane + "_to_" + airport

def Fly(plane: str, airport_from: str, airport_to: str) -> str:
    return "fly_" + plane + "_from_" + airport_from + "_to_" + airport_to


boolean: set[bool] = {True, False}
null: set[str] = {'null'}
def create_air_cargo_transportation(cargo: set[str], planes: set[str], airports: set[str]) -> STRIPS_domain:
    #feature_domain_dict = {Cargo(x):True for x in cargo}
    #feature_domain_dict.update({Cargo(x):False for x in cargo_storage})
    #feature_domain_dict.update({Plane(x):True for x in planes})
    #feature_domain_dict.update({Plane(x):False for x in cargo|airports})
    #feature_domain_dict.update({Airport(x):True for x in airports})
    #feature_domain_dict.update({Airport(x):False for x in cargo|planes})
    feature_domain_dict: dict[str, set[str]] = {In(x): planes | null for x in cargo}
    feature_domain_dict.update({At(x): airports | null for x in planes | cargo})


    stmap: set[Strips] = {Strips(Load(c, p, a), {At(c): a, At(p): a}, {In(c): p, At(c): 'null'})
             for c in cargo
             for p in planes
             for a in airports}
    stmap.update({Strips(Unload(c, p, a), {In(c): p, At(p): a}, {At(c): a, In(c): 'null'})
                  for c in cargo
                  for p in planes
                  for a in airports})

    stmap.update({Strips(Fly(p, a_from, a_to), {At(p): a_from}, {At(p): a_to})
                  for p in planes
                  for a_from in airports
                  for a_to in airports
                  if a_from != a_to})

    return STRIPS_domain(feature_domain_dict, stmap)

In [8]:
from stripsProblem import Planning_problem
# Problem 1

air_cargo1dom: STRIPS_domain = create_air_cargo_transportation(cargo={'c1','c2','c3'}, planes={'p1','p2'}, airports={'sfo', 'jfk', 'lax'})

air_setup1: dict[str, str] = {
    At('c1'):'sfo',
    At('c2'):'jfk',
    At('c3'):'lax',
    At('p1'):'sfo',
    At('p2'):'jfk',
    In('c1'):'null',
    In('c2'):'null',
    In('c3'):'null',
}


air_cargo1 = Planning_problem(air_cargo1dom,
                              air_setup1, #initial state
                              {At('c1'):'jfk', At('c2'):'sfo', In('c1'):'null', In('c2'):'null'}) #goal


In [9]:
from time import perf_counter

In [10]:
#test
from searchMPP import SearcherMPP
from stripsForwardPlanner import Forward_STRIPS

print('Problem 1 - No heuristic:')

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo1)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 1', False, end - start]

Problem 1 - No heuristic:
Solution: {'c1_is_at': 'sfo', 'c2_is_at': 'jfk', 'c3_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'null', 'c3_is_in': 'null'}
   --load_c2_from_jfk_to_p2--> {'c1_is_at': 'sfo', 'c2_is_at': 'null', 'c3_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'p2', 'c3_is_in': 'null'}
   --fly_p2_from_jfk_to_sfo--> {'c1_is_at': 'sfo', 'c2_is_at': 'null', 'c3_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'sfo', 'c1_is_in': 'null', 'c2_is_in': 'p2', 'c3_is_in': 'null'}
   --unload_c2_from_p2_to_sfo--> {'c1_is_at': 'sfo', 'c2_is_at': 'sfo', 'c3_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'sfo', 'c1_is_in': 'null', 'c2_is_in': 'null', 'c3_is_in': 'null'}
   --load_c1_from_sfo_to_p2--> {'c1_is_at': 'null', 'c2_is_at': 'sfo', 'c3_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'sfo', 'c1_is_in': 'p2', 'c2_is_in': 'null', 'c3_is_in': 'null'}
   --fly_p2_from_sfo_to_jfk--> {'c1_is_at': 'null', 'c2_is_at

In [11]:
def h1(assignment, goal) -> int:
    """Number of mismatched assignments between assignment and goal."""
    return sum(1 for k, v in goal.items() if assignment[k] != v)

In [12]:
print('Problem 1 - With heuristic:')

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(air_cargo1, h1)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 1', True, end - start]

Problem 1 - With heuristic:
Solution: {'c1_is_at': 'sfo', 'c2_is_at': 'jfk', 'c3_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'null', 'c3_is_in': 'null'}
   --load_c2_from_jfk_to_p2--> {'c1_is_at': 'sfo', 'c2_is_at': 'null', 'c3_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'p2', 'c3_is_in': 'null'}
   --fly_p2_from_jfk_to_sfo--> {'c1_is_at': 'sfo', 'c2_is_at': 'null', 'c3_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'sfo', 'c1_is_in': 'null', 'c2_is_in': 'p2', 'c3_is_in': 'null'}
   --unload_c2_from_p2_to_sfo--> {'c1_is_at': 'sfo', 'c2_is_at': 'sfo', 'c3_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'sfo', 'c1_is_in': 'null', 'c2_is_in': 'null', 'c3_is_in': 'null'}
   --load_c1_from_sfo_to_p2--> {'c1_is_at': 'null', 'c2_is_at': 'sfo', 'c3_is_at': 'lax', 'p1_is_at': 'sfo', 'p2_is_at': 'sfo', 'c1_is_in': 'p2', 'c2_is_in': 'null', 'c3_is_in': 'null'}
   --fly_p2_from_sfo_to_jfk--> {'c1_is_at': 'null', 'c2_is_

In [13]:
# Problem 2
import stripsProblem

print("Problem 2 - No heuristic:")

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(stripsProblem.problem1)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 2', False, end - start]

Problem 2 - No heuristic:
Solution: {'RLoc': 'lab', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mcc_lab--> {'RLoc': 'off', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mcc_off--> {'RLoc': 'cs', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --puc--> {'RLoc': 'cs', 'MW': True, 'SWC': True, 'RHC': True, 'RHM': False}
   --mc_cs--> {'RLoc': 'off', 'MW': True, 'SWC': True, 'RHC': True, 'RHM': False}
   --dc--> {'RLoc': 'off', 'MW': True, 'SWC': False, 'RHC': False, 'RHM': False} (cost: 5)
 17 paths have been expanded and 14 paths remain in the frontier
Elapsed time: 0.000709 seconds


In [14]:
print("Problem 2 - With heuristic:")

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(stripsProblem.problem1, h1)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 2', True, end - start]

Problem 2 - With heuristic:
Solution: {'RLoc': 'lab', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mcc_lab--> {'RLoc': 'off', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mcc_off--> {'RLoc': 'cs', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --puc--> {'RLoc': 'cs', 'MW': True, 'SWC': True, 'RHC': True, 'RHM': False}
   --mc_cs--> {'RLoc': 'off', 'MW': True, 'SWC': True, 'RHC': True, 'RHM': False}
   --dc--> {'RLoc': 'off', 'MW': True, 'SWC': False, 'RHC': False, 'RHM': False} (cost: 5)
 11 paths have been expanded and 10 paths remain in the frontier
Elapsed time: 0.000565 seconds


In [15]:
# Problem 3

print("Problem 3 - No heuristic:")

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(stripsProblem.problem2)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 3', False, end - start]

Problem 3 - No heuristic:
Solution: {'RLoc': 'lab', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mc_lab--> {'RLoc': 'mr', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --pum--> {'RLoc': 'mr', 'MW': False, 'SWC': True, 'RHC': False, 'RHM': True}
   --mc_mr--> {'RLoc': 'cs', 'MW': False, 'SWC': True, 'RHC': False, 'RHM': True}
   --puc--> {'RLoc': 'cs', 'MW': False, 'SWC': True, 'RHC': True, 'RHM': True}
   --mc_cs--> {'RLoc': 'off', 'MW': False, 'SWC': True, 'RHC': True, 'RHM': True}
   --dc--> {'RLoc': 'off', 'MW': False, 'SWC': False, 'RHC': False, 'RHM': True}
   --dm--> {'RLoc': 'off', 'MW': False, 'SWC': False, 'RHC': False, 'RHM': False} (cost: 7)
 26 paths have been expanded and 18 paths remain in the frontier
Elapsed time: 0.001219 seconds


In [16]:
print("Problem 3 - With heuristic:")

start: float = perf_counter()
SearcherMPP(Forward_STRIPS(stripsProblem.problem2, h1)).search()
end: float = perf_counter()

print(f'Elapsed time: {end - start:.6f} seconds')

data.loc[len(data)] = ['Problem 3', True, end - start]

Problem 3 - With heuristic:
Solution: {'RLoc': 'lab', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mc_lab--> {'RLoc': 'mr', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --pum--> {'RLoc': 'mr', 'MW': False, 'SWC': True, 'RHC': False, 'RHM': True}
   --mc_mr--> {'RLoc': 'cs', 'MW': False, 'SWC': True, 'RHC': False, 'RHM': True}
   --puc--> {'RLoc': 'cs', 'MW': False, 'SWC': True, 'RHC': True, 'RHM': True}
   --mc_cs--> {'RLoc': 'off', 'MW': False, 'SWC': True, 'RHC': True, 'RHM': True}
   --dm--> {'RLoc': 'off', 'MW': False, 'SWC': True, 'RHC': True, 'RHM': False}
   --dc--> {'RLoc': 'off', 'MW': False, 'SWC': False, 'RHC': False, 'RHM': False} (cost: 7)
 18 paths have been expanded and 18 paths remain in the frontier
Elapsed time: 0.000822 seconds


In [17]:
data

,problem,used_heuristic,elapsed_time
0,Problem 1,False,0.093868
1,Problem 1,True,0.027656
2,Problem 2,False,0.000709
3,Problem 2,True,0.000565
4,Problem 3,False,0.001219
5,Problem 3,True,0.000822


In [18]:
data.dtypes

problem            object
used_heuristic       bool
elapsed_time      float64
dtype: object